 ## Trabajo N.º 3 
## Integración de árboles de decisión y clustering jerárquico en la modelación de patrones sísmicos en Chile


### Abstract

Este trabajo integra y extiende los análisis realizados en las Evaluaciones 1 y 2 sobre la actividad sísmica en Chile, utilizando el mismo conjunto de datos de eventos registrados por el Centro Sismológico Nacional entre 2012 y 2025. El objetivo central es comparar el rendimiento e interpretabilidad de distintos modelos de regresión, incorporar árboles de decisión tanto regresivos como lógicos, y aplicar técnicas de clustering jerárquico para identificar patrones de agrupamiento en la magnitud, profundidad y localización de los sismos.

A partir de las variables numéricas principales (Latitude, Longitude, Depth, Magnitude) y de la etiqueta derivada zona_sismica, se construyen y evalúan modelos de regresión lineal y basados en árboles, así como un árbol de decisión de clasificación y clústeres jerárquicos en variantes aditiva y divisiva. Los resultados permiten contrastar los modelos de las evaluaciones anteriores, visualizar reglas de decisión explícitas y descubrir grupos de eventos con comportamientos similares. Finalmente, se formulan conclusiones y recomendaciones de negocio orientadas a mejorar la interpretación de los patrones sísmicos y apoyar la priorización de zonas de vigilancia.


## 1. Introducción

Chile es uno de los países con mayor actividad sísmica del mundo, lo que convierte a la medición, análisis y comunicación de los sismos en un problema permanente de seguridad y de gestión del riesgo. En las Evaluaciones 1 y 2 se trabajó con un conjunto de datos del Centro Sismológico Nacional, explorando la distribución de magnitudes y profundidades, comparando macrozonas y desarrollando modelos de regresión y clasificación básicos.

En esta tercera entrega se retoma el mismo dataset, pero con un foco más integrado y reflexivo. Nuestro interés no es solo aplicar nuevas técnicas, sino también comparar lo realizado anteriormente y preguntarse qué modelos aportan mayor capacidad de explicación y apoyo a la toma de decisiones. Este trabajo busca profundizar la comprensión de cómo variables como latitud, longitud, profundidad y magnitud se relacionan entre sí, y cómo los modelos pueden ayudar a mejorar la detección de patrones relevantes para la gestión sísmica.

La integración de árboles de decisión y clustering jerárquico permite avanzar hacia modelos más interpretables, donde las reglas y los agrupamientos puedan explicarse de manera sencilla a usuarios no técnicos, manteniendo al mismo tiempo una base cuantitativa sólida.


## 3. Equipos, software y librerías utilizadas

### 3.1 Equipo de trabajo

El análisis se realizó en computadores con las siguientes características relevantes para el procesamiento de datos:

- Procesador Intel Core i7
- Memoria RAM 16 GB 
- Unidad de almacenamiento sólido (SSD) de 512 GB y 1 TB
- Tarjetas de video: Intel UHD Graphics Xe (integrada) y NVIDIA GeForce RTX 3060
- Sistema operativo Windows 10 de 64 bits.

Estas especificaciones permiten trabajar de forma fluida con notebooks de Python, manejar dataframes de tamaño medio y ejecutar modelos de Machine Learning de complejidad moderada sin problemas de rendimiento.

### 3.2 Software, entorno de desarrollo y trabajo colaborativo

- Lenguaje: Python 3.12.10.
- Entorno de desarrollo: Visual Studio Code con la extensión de Jupyter Notebook.
- Sistema de control de versiones: Git y GitHub.
- Cliente de escritorio: GitHub Desktop para sincronizar el repositorio entre distintos computadores de trabajo.
- Estructura de carpetas:
  - data/: contiene el archivo seismic_data.csv con los registros sísmicos.
  - figuras/: almacena las imágenes generadas por los gráficos del análisis.

El proyecto se gestionó mediante un repositorio en GitHub, utilizando GitHub Desktop para realizar commit, push y pull de los cambios entre equipos. Visual Studio Code se utilizó como editor principal para desarrollar el notebook, gestionar ramas cuando fue necesario y revisar el historial de versiones, lo que facilitó el trabajo colaborativo.

### 3.3 Librerías principales

Las librerías de Python más utilizadas en este trabajo son:

- pandas y numpy para manipulación y preparación de datos.
- matplotlib y seaborn para visualización de gráficos descriptivos y comparativos.
- scikit-learn para construcción y evaluación de modelos de regresión, árboles de decisión, clasificación y clustering jerárquico.
- nbformat e ipykernel como soporte para la ejecución reproducible del notebook.

Las versiones específicas de estas dependencias se encuentran documentadas en el archivo requirements.txt del proyecto.



In [ ]:
# =============================================================================
# 4. CONFIGURACIÓN GENERAL DEL PROYECTO
# =============================================================================

import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Estilo de gráficos y opciones de Pandas
plt.style.use("default")
sns.set(style="whitegrid")
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: f"{x:,.4f}")

# Directorios base
DATA_DIR = "data"
FIG_DIR = "figuras"

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(FIG_DIR, exist_ok=True)

def guardar_figura(fig, nombre):
    """
    Guarda una figura en la carpeta 'figuras' con nombre <nombre>.png
    """
    ruta = os.path.join(FIG_DIR, f"{nombre}.png")
    fig.savefig(ruta, dpi=300, bbox_inches="tight")
    print(f"Figura guardada en: {ruta}")


In [ ]:
# =============================================================================
# 5. CARGA DEL DATASET SÍSMICO
# =============================================================================

NOMBRE_ARCHIVO = "seismic_data.csv"
ruta_csv = os.path.join(DATA_DIR, NOMBRE_ARCHIVO)

if not os.path.exists(ruta_csv):
    raise FileNotFoundError(
        f"No se encontró el archivo '{ruta_csv}'. "
        "Verifique que el CSV esté dentro de la carpeta 'data/'."
    )

print(f"Cargando dataset desde: {ruta_csv}")
df = pd.read_csv(ruta_csv)

print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")
display(df.head())


In [ ]:
# =============================================================================
# 6. PREPARACIÓN INICIAL DE VARIABLES
# =============================================================================

# Conversión de fecha a tipo datetime
df["Date"] = pd.to_datetime(df["Date(UTC)"], errors="coerce")

df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.to_period("M").astype(str)
df["Day"] = df["Date"].dt.day

def zona_por_lat(lat):
    """
    Asigna una macrozona sísmica aproximada según la latitud:
    - Norte: lat >= -23
    - Centro: -38 <= lat < -23
    - Sur: lat < -38
    """
    if pd.isna(lat):
        return np.nan
    if lat >= -23:
        return "Norte"
    elif lat <= -38:
        return "Sur"
    else:
        return "Centro"

df["zona_sismica"] = df["Latitude"].apply(zona_por_lat)

print("Distribución de la variable 'zona_sismica':")
print(df["zona_sismica"].value_counts(dropna=False))
display(df[["Date", "Latitude", "Longitude", "Depth", "Magnitude", "zona_sismica"]].head())
